In [28]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

mmseqs_result = "../mmseqs_mutated_sequences/results.csv"
mmseqs_result = "../mmseqs_subset_1000/results.csv"
headers = ["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits"]

In [8]:
df = pd.read_csv(mmseqs_result, sep="\t")
df.head(2)

,query,target,fident,evalue,bits
0,P42212,SI12V,0.997,9.731000e-165,504
1,P42212,P42212,1.000,9.731000e-165,504


In [29]:
# Get unique sequence IDs and create mapping to integers
all_seqs = sorted(list(set(df['query'].unique()) | set(df['target'].unique())))
seq_to_idx = {seq: idx for idx, seq in enumerate(all_seqs)}

# Convert sequences to integer indices
row_idx = np.array([seq_to_idx[seq] for seq in df['query']])
col_idx = np.array([seq_to_idx[seq] for seq in df['target']])

# Create sparse matrix in COO format
n = len(all_seqs)
sparse_matrix = coo_matrix(
    (df["fident"], (row_idx, col_idx)),
    shape=(n, n)
)

# Convert to dense matrix
dense_matrix = sparse_matrix.toarray()

# Convert to DataFrame with proper labels
matrix = pd.DataFrame(
    dense_matrix,
    index=all_seqs,
    columns=all_seqs
)

In [32]:
count_zeros = np.count_nonzero(matrix == 0)
count_zeros

0

In [36]:
output_file = "../processed_data/similarity_matrix.csv"
matrix.to_csv(output_file, sep=',')